In [65]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import image
import seaborn as sns
import cv2 as cv
import tensorflow as tf
from tensorflow.keras import layers, models
import functools as f
from sklearn.model_selection import train_test_split


%matplotlib.inline

UsageError: Line magic function `%matplotlib.inline` not found.


In [66]:
#896 Mild_Demented
#64 Moderate_Demented
#3200 Non_Demented
#2240 Very_Mild_Demented

In [67]:
mild_dem = ('mild_dem '*896).split()
mod_dem = ('mod_dem '*64).split()
non_dem = ('non_dem '*3200).split()
very_mild_dem = ('very_mild_dem '*2240).split()

In [68]:
labels = [mild_dem, mod_dem, non_dem, very_mild_dem]
labels = f.reduce(lambda x, y: x+y, labels)
labels = pd.Series(labels)
labels = pd.DataFrame(labels, columns = ['label'])

In [69]:
labels.to_csv('labels.csv')

In [71]:
y = labels
y_train, y_test = train_test_split(y, stratify=y)